In [ ]:
#!pip install googletrans
# 이 버전으로 설치 후 번역 기능을 실행할 때
#(AttributeError: 'NoneType' object has no attribute 'group') 에러 발생
# 3.0.0 버전까지 AttributeError: 'NoneType' object has no attribute 'group' 오류가 발생하여 실행이 잘 되지 않는 이슈가 있었기때문에 4.0 버전을 지정하여 설치

#※ Googletrans API

## 1) 주의점
 - 공식 Google 번역 API를 사용하지 않고 우회하여 translate.google.com 를 호출하여 사용함.
 - 때문에 비정상적인 호출에 대하여 API가 구글에 의해서 Ban이 당할 수 있으나 빠르게 구현하여 사용할 수 있다는 것

## 2) 특징
- 언어 감지 및 번역을 도와주는 라이브러리
- 빠르고 안정적이다. translate.google.com에서 사용하는 것과 동일한 서버를 사용
- 자동 언어 감지
- 대량 번역
- 맞춤형 서비스 URL
- 연결 풀링(requests.Session 사용의 이점)
- HTTP/2 지원

## 3) 라이센스 및 제한
- Googletrans는 Google 번역 API를 구현한 무료 무제한 파이썬 라이브러리
- Google 번역 Ajax API를 사용하여 감지 및 번역과 같은 메소드를 호출

## 4) 참고
- 단일 텍스트의 최대 글자 수는 15k
- Google 번역 웹 버전의 제한으로 인해 이 API는 라이브러리가 항상 제대로 작동한다고 보장하지 않음.
- 번역할 텍스트의 리스트가 많으면 반복문을 활용하여 호출 시 오래 걸려서 만족한 응답시간을 기대할 수 없음.
- 반복적으로 호출 시 내부적으로는 Googletrans API를 여러 번 호출 하기 때문에 오래 걸림.
- 대용량 데이터 번역에는 유용하지 않음.

## 5) 출처
- https://yscho03.tistory.com/m/93

In [ ]:
!pip3 uninstall googletrans # 기존 설치 버전 삭제
#!pip3 install googletrans==3.1.0a0
!pip install googletrans==4.0.0-rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 6.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17414 sha256=c1a6642b3d01f35d492124dc484b3ff34e8c627a436b10d52265ba17b8a779b4
  Stored in directory: /root/.cache/pip/wheels/a9/25/af/715361fa79594524c89c75d293def652045f3fdce6ca398712
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0


## Import libraries

In [ ]:
from googletrans import Translator
import pandas as pd
import random
import copy

## Data Preprocessing

In [ ]:
# API 요청 횟수 줄임으로써 응답시간 줄이기
'''
15k까지 텍스트 제한이 걸리므로 리스트 요소마다 구분자를 넣어주고,
15k씩 텍스트를 잘라서 요청하여 API 요청 횟수를 줄여 네트워크 부하를 줄임.
시행착오
한글 > 영어: MAX_BYTE = 1024*15 로 설정하여 시행
영어 > 한글: MAX_BYTE = 5000 로 설정하여 시행
이상 혹은 이하로 설정 시 (the JSON object must be str, bytes or bytearray, not NoneType), The read operation timed out 에러 발생
'''
DELIMITER = '\n'
MAX_BYTE = 5000  # 15k, 5k, 4.5k, 4k

def wrap_text(texts):
    """
    번역을 하지 않는 구간을 특정 기호로 문장을 합쳐서 요청
    예) ['안녕하시오', '잘 가시오?'] → '안녕하시오.\n잘 가시오.'
    """
    texts = [text.replace(DELIMITER, '<span class="notranslate">space</span>') for text in texts]
    return DELIMITER.join(texts)

def unwrap_text(wrapped_text):
    """
    번역된 문장을 다시 특정 기호로 문장을 잘라줌.
    예) '안녕하시오.\n잘 가시오.' →  ['안녕하시오', '잘 가시오.']
    """
    texts = wrapped_text.split(DELIMITER)
    texts = [text.replace('<span class="notranslate">space</span>', DELIMITER) for text in texts]
    return texts
    
def chunk_text_byte(text, max_byte=MAX_BYTE, delimiter=DELIMITER):
    """
    대량의 번역 문장들을 순차적으로 translate 함수를 요청시 내부적으로는 API 요청을 
    여러번 요청하게 되어 있으므로 성능적으로 느린 응답을 가져오게 되므로
    15K 까지 글자수가 제한이 되어있으므로 15K씩 잘라서 요청하는 것으로 네트워크 부하를 줄일 수 있음.
    """
    text = text.encode('utf-8')
    delimiter = delimiter.encode('utf-8')
    words = iter(text.split(delimiter))
    lines, current = [], next(words)

    for word in words:
        if len(current) + 1 + len(word) > max_byte:
            lines.append(current.decode('utf-8'))
            current = word
        else:
            current += delimiter + word
    lines.append(current.decode('utf-8'))
    return lines

In [ ]:
dt = pd.read_csv('/content/drive/MyDrive/Team_Project/Project3(translate)/구어체_test.csv') #encoding='utf-8'
dt

In [ ]:
# 데이터 열을 선택하여 리스트 화
dt = dt['원문']
dt_val = dt.values
dt_list = dt_val.tolist()

In [ ]:
print(len(dt_list))

100000


In [ ]:
'''
translator = Translator()
translate_texts = []

# 100,000개 데이터를 한 번에 번역시 글자 15k 제한으로 인해 The read operation timed out 에러 발생
for target_text in dt_list_temp:
    translate_texts.append(translator.translate(text=target_text, dest='en').text)
'''

In [ ]:
translated_texts = []

In [ ]:
# 여비 샘플 깊은 복사 실행
dt_list_temp = copy.deepcopy(dt_list)

In [ ]:
# 80 * 1250 = 100,000
for _ in range(1, 1251):

    dt_test = []
    # 80으로 설정한 이유: 한글 > 영어 번역시 80초과으로 설정하면 번역되지 않음.
    for _ in range(80):
        dt_test.append(dt_list_temp[0])
        dt_list_temp.pop(0)

    translator = Translator()
    translation = ''

    for splitted_text in chunk_text_byte(wrap_text(dt_test)):
        translation += translator.translate(text=splitted_text, dest='en').text

    for translated_text in unwrap_text(translation):
        translated_texts.append(translated_text)

In [ ]:
dt_t_list = pd.DataFrame(dt_list, columns = ['번역전'])
dt_t_list['영어'] = translated_texts
#dt_ko = pd.DataFrame(translated_texts, columns = ['영어번역'])

In [ ]:
dt_t_list.to_csv('/content/drive/MyDrive/Team_Project/Project3(translate)/trans_test(1).csv', encoding='utf-8', index=False)

In [ ]:
print(len(dt_list_temp))
print(len(dt_test))
print(len(translated_texts))

0
240
100000


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Team_Project/Project3(translate)/trans_test(1).csv') #encoding='utf-8'

In [ ]:
df_t = df['영어']
df_val = df_t.values
df_list = df_val.tolist()

In [ ]:
print(len(df_list))

100000


In [ ]:
df_trans_temp = copy.deepcopy(df_list)

In [ ]:
'''translated_texts_ko = []'''

In [ ]:
translated_texts_ko_t = []

In [ ]:
for _ in range(1, 1251):

    df_test = []
    for _ in range(80):
        df_test.append(df_trans_temp[0])
        df_trans_temp.pop(0)

    translator = Translator()
    translation_ko = ''

    if df_test is not None:
        for splitted_text in chunk_text_byte(wrap_text(df_test)):
            translation_ko += translator.translate(text=splitted_text, dest='ko').text

    for translated_text in unwrap_text(translation_ko):
        translated_texts_ko_t.append(translated_text)

In [ ]:
print(len(df_trans_temp))
print(len(df_test))
print(len(translated_texts_ko_t))

0
80
99988


In [ ]:
translated_texts_ko_t

In [ ]:
df_t_list = pd.DataFrame(translated_texts_ko_t, columns = ['번역후'])
df_t_list.to_csv('/content/drive/MyDrive/Team_Project/Project3(translate)/trans_test(3).csv', index=False)

In [ ]:
df_t_list

In [ ]:
'''#dt_t = pd.DataFrame(dt_list, columns = ['구어체'])
dt_en = pd.DataFrame(dt_test_trans, columns = ['영어번역'])
dt_ko = pd.DataFrame(dt_test_trans_ko, columns = ['한글번역'])'''

In [ ]:
trans_dt = pd.read_csv('/content/drive/MyDrive/Team_Project/Project3(translate)/trans_test_dastset(1).csv', encoding='cp949')

In [ ]:
trans_dt

,번역전,영어번역,한글번역
0,사과는 잘 씻은 뒤 껍질 채 먹는 게 좋다네요.,It is better to wash the apples and eat it.,사과를 씻고 먹는 것이 낫습니다.
1,내가 언제까지 거기 가면 됩니까?,How long can I go there?,거기에 얼마나 오래 갈 수 있습니까?
2,우리가 작성해서 전달해 드려야 하나요?,Should we write and deliver?,우리는 쓰고 전달해야합니까?
3,"난 지금 행복해요, 세상이 나를 원하고 있죠.","I'm happy now, the world wants me.",나는 지금 행복합니다. 세상은 나를 원합니다.
4,그는 틀림없이 꽃 가게에 들렸을 겁니다.,He must have been in a flower shop.,그는 꽃 가게에 있었을 것입니다.
...,...,...,...
99995,그럼 컨셉을 약간 바꾸는 것도 좋은 방법이겠네요.,Then it's a good idea to change the concept a ...,그런 다음 개념을 조금 바꾸는 것이 좋습니다.
99996,객실 안에 비밀번호가 작성되어 있습니다.,A password is written in the room.,암호는 방에 작성됩니다.
99997,4번 게이트로 갔을 때 바로 찾을 수 있나요?,Can I find it right when I went to Gate 4?,게이트 4에 갔을 때 바로 찾을 수 있습니까?
99998,지금 3개나 빠진 상태인데 한꺼번에 이렇게 많이 빠져도 상관없는 건가요?,I'm missing 3 of them right now. Does it matte...,지금 3개를 놓치고 있습니다. 이만큼 한꺼번에 잃어도 상관없나요?


In [ ]:
trans_dt.to_csv('/content/drive/MyDrive/Team_Project/Project3(translate)/trans_test_dastset(2).csv', encoding='utf-8', index=False)